In [1]:
# Test Oracle Connection and Django Compatibility
import os
import sys

# Add the project directory to Python path
project_path = r'c:\Users\iP\Full Stack\السلاح backend'
sys.path.insert(0, project_path)

# Set Django settings
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'weaponpowercloud_backend.settings')
os.environ.setdefault('USE_ORACLE', 'False')  # Keep SQLite for testing initially

import django
django.setup()

# Test Django models
from authentication.models import User
from surveys.models import Survey, Question

print("✅ Django setup completed successfully!")
print("✅ Models imported without errors!")

# Test Oracle-compatible managers
print("\n📋 Testing Oracle-compatible managers:")
try:
    # Test user manager
    print(f"User manager: {User.objects.__class__.__name__}")
    
    # Test survey manager
    print(f"Survey manager: {Survey.objects.__class__.__name__}")
    
    # Test question manager  
    print(f"Question manager: {Question.objects.__class__.__name__}")
    
    print("✅ All managers loaded successfully!")
except Exception as e:
    print(f"❌ Manager test failed: {e}")

# Test hash field generation
print("\n🔐 Testing hash field generation:")
try:
    import hashlib
    
    test_email = "test@example.com"
    test_username = "testuser"
    test_title = "Test Survey"
    
    email_hash = hashlib.sha256(test_email.encode('utf-8')).hexdigest()
    username_hash = hashlib.sha256(test_username.encode('utf-8')).hexdigest()
    title_hash = hashlib.sha256(test_title.encode('utf-8')).hexdigest()
    
    print(f"Email hash: {email_hash[:16]}...")
    print(f"Username hash: {username_hash[:16]}...")
    print(f"Title hash: {title_hash[:16]}...")
    
    print("✅ Hash generation working correctly!")
except Exception as e:
    print(f"❌ Hash generation failed: {e}")

✅ Django setup completed successfully!
✅ Models imported without errors!

📋 Testing Oracle-compatible managers:
User manager: OracleCompatibleUserManager
Survey manager: OracleCompatibleSurveyManager
Question manager: OracleCompatibleQuestionManager
✅ All managers loaded successfully!

🔐 Testing hash field generation:
Email hash: 973dfe463ec85785...
Username hash: ae5deb822e0d7199...
Title hash: 04da1dee32de3494...
✅ Hash generation working correctly!


In [1]:
import oracledb
from contextlib import closing

# ---------- 1) تهيئة الاتصال ----------
USERNAME = 'APEX_PUBLIC_USER'
PASSWORD = 'KgJyrx3$1'
HOST     = '185.197.251.203' 
PORT     = 1521
SERVICE  = 'PROD'                 

dsn = oracledb.makedsn(HOST, PORT, service_name=SERVICE)

# ---------- 2) فتح الاتصال واختباره ----------
try:
    with closing(oracledb.connect(user=USERNAME, password=PASSWORD, dsn=dsn)) as conn, \
         closing(conn.cursor()) as cur:

        # جرّب استعلامًا بسيطًا
        cur.execute('SELECT 1 FROM dual')
        print(f'Connection OK, result = {cur.fetchone()[0]}')

        # ---------- 3) إنشاء جدول تجريبي ----------
        cur.execute("""
            CREATE TABLE test_conn (
                id         NUMBER GENERATED BY DEFAULT AS IDENTITY,
                name       VARCHAR2(50),
                created_at DATE DEFAULT SYSDATE
            )
        """)
        print('Table created.')

        # إدخال صف
        cur.execute("INSERT INTO test_conn (name) VALUES (:1)", ["sample"])
        conn.commit()
        print('Row inserted.')

        # الاستعلام للتأكد
        cur.execute("SELECT id, name, created_at FROM test_conn")
        print('Current rows:', cur.fetchall())

        # ---------- 4) حذف الصف ----------
        cur.execute("DELETE FROM test_conn WHERE name = :1", ["sample"])
        conn.commit()
        print('Row deleted.')

        # ---------- 5) إسقاط الجدول ----------
        cur.execute("DROP TABLE test_conn PURGE")
        print('Table dropped, cleanup complete.')

except oracledb.DatabaseError as err:
    error, = err.args
    print(f'Oracle-Error-Code: {error.code}\nOracle-Error-Message: {error.message}')


Connection OK, result = 1
Table created.
Row inserted.
Current rows: [(1, 'sample', datetime.datetime(2025, 8, 7, 23, 59, 36))]
Row deleted.
Table dropped, cleanup complete.
